In [19]:
#importing libraries

from bs4 import BeautifulSoup
from requests import get
from time import sleep
import datetime
import numpy as np
import smtplib

In [20]:
seasons = np.arange(1, 7, 1)

#initializing empty list to store variables scraped
episodes = []
names = []
releaseDates = []
scores = []
votes = []


# https://httpbin.org/ge link  to user-agent
# , "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

#getting content
for season in seasons:
    response = get("https://www.imdb.com/title/tt3032476/episodes?season="
                  +str(season), headers=headers
                  )
    sleep(1)

    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    # Parse HTML Code
    soup1 = BeautifulSoup(response.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    episode_containers1 = soup2.find_all('div', class_ = 'list_item odd')
    episode_containers2 = soup2.find_all('div', class_ = 'list_item even')
    episode_containers = episode_containers1+episode_containers2
    # Running loop for every episode in the page
    for container in episode_containers:
        
        #episode
        if container.find('div', class_ = 'info') is not None:
            episode = container.div.div.text.strip() # inside container find div -> div -> get text > strip
            episodes.append(episode)
        else:
            episodes.append(None)
        
        
        #names
        if container.find('div', class_ = 'info') is not None:
            name = container.find('div', class_ = 'info').strong.text.strip()
            names.append(name)
        else:
            names.append(None)
        
        
        #releaseDates    
        if container.find('div', class_ = 'info').find('div', class_ = 'airdate') is not None:   
            releaseDate = container.find('div', class_ = 'info').find('div', class_ = 'airdate').text.strip()
            releaseDates.append(releaseDate)
        else:
            releaseDates.append(None)
        
        
        #scores
        if container.find('div', class_ = 'info').find('span', class_ = 'ipl-rating-star__rating') is not None:  
            score = container.find('div', class_ = 'info').find('span', class_ = 'ipl-rating-star__rating').text.strip()
            scores.append(score)
        else:
            scores.append(None)
            
            
        #votes
        if container.find('div', class_ = 'info').find('span', class_ = 'ipl-rating-star__total-votes') is not None:  
            vote = container.find('div', class_ = 'info').find('span', class_ = 'ipl-rating-star__total-votes').text.strip()
            votes.append(vote)
        else:
            votes.append(None)


In [21]:
import pandas as pd

BCS_df = pd.DataFrame({'episode_title': names,
                       'release_Date': releaseDates,
                       'episode': episodes,
                       'imdb_score': scores,
                       'user_votes': votes
    
})


In [38]:
BCS_df

,episode_title,release_Date,episode,imdb_score,user_votes
0,Uno,2015-02-08,"S1, Ep1",8.5,"(19,901)"
1,Nacho,2015-02-16,"S1, Ep3",8.7,"(16,003)"
2,Alpine Shepherd Boy,2015-03-02,"S1, Ep5",8.0,"(13,982)"
3,Bingo,2015-03-16,"S1, Ep7",8.6,"(13,350)"
4,Pimento,2015-03-30,"S1, Ep9",9.4,"(17,670)"
...,...,...,...,...,...
58,Hit and Run,2022-05-02,"S6, Ep4",8.6,"(14,990)"
59,Axe and Grind,2022-05-16,"S6, Ep6",8.2,"(14,114)"
60,Point and Shoot,2022-07-11,"S6, Ep8",9.7,"(33,715)"
61,Nippy,2022-07-25,"S6, Ep10",8.5,"(21,311)"


In [39]:
BCS_df.to_csv('BCS_IMDb_WebScrapped_Data.csv', index=False)
df = pd.read_csv(r'C:\Users\burun\BCS_IMDb_WebScrapped_Data.csv')
print(df)

          episode_title release_Date   episode  imdb_score user_votes
0                   Uno   2015-02-08   S1, Ep1         8.5   (19,901)
1                 Nacho   2015-02-16   S1, Ep3         8.7   (16,003)
2   Alpine Shepherd Boy   2015-03-02   S1, Ep5         8.0   (13,982)
3                 Bingo   2015-03-16   S1, Ep7         8.6   (13,350)
4               Pimento   2015-03-30   S1, Ep9         9.4   (17,670)
..                  ...          ...       ...         ...        ...
58          Hit and Run   2022-05-02   S6, Ep4         8.6   (14,990)
59        Axe and Grind   2022-05-16   S6, Ep6         8.2   (14,114)
60      Point and Shoot   2022-07-11   S6, Ep8         9.7   (33,715)
61                Nippy   2022-07-25  S6, Ep10         8.5   (21,311)
62           Waterworks   2022-08-08  S6, Ep12         9.5   (24,396)

[63 rows x 5 columns]


In [37]:
BCS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   episode_title  63 non-null     object        
 1   release_Date   63 non-null     datetime64[ns]
 2   episode        63 non-null     object        
 3   imdb_score     63 non-null     float64       
 4   user_votes     63 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 2.6+ KB


In [42]:
BCS_df['release_Date'] = BCS_df['release_Date'].astype('datetime64')
BCS_df['imdb_score'] = BCS_df['imdb_score'].astype('float')
